In [1]:
import pandas as pd
from tqdm import tqdm
import os
import numpy as np
import pickle

#### Window 저장경로 > Mac 저장경로 변환

In [2]:
meta_data = pd.read_csv("/opt/ml/fashion-image-search/storage/csv/Top.csv")
meta_data

,key,link,name,image_link,path
0,1000696,https://www.musinsa.com/app/goods/1000696,어커버(ACOVER) [세트상품]포인트 오버핏 롱티셔츠,https://image.msscdn.net/images/goods_img/2019...,/opt/ml/fashion-image-search/storage/images/To...
1,1001714,https://www.musinsa.com/app/goods/1001714,엠엠엘지(MMLG) [Mmlg] MMLG HF-T (BLACK),https://image.msscdn.net/images/goods_img/2019...,/opt/ml/fashion-image-search/storage/images/To...
2,1008039,https://www.musinsa.com/app/goods/1008039,아노블리어(ANOBLIR) 루즈핏 오픈카라 텐셀 린넨 셔츠 [오트밀],https://image.msscdn.net/images/goods_img/2019...,/opt/ml/fashion-image-search/storage/images/To...
3,1008043,https://www.musinsa.com/app/goods/1008043,아노블리어(ANOBLIR) 루즈핏 오픈카라 텐셀 린넨 셔츠 [네이비],https://image.msscdn.net/images/goods_img/2019...,/opt/ml/fashion-image-search/storage/images/To...
4,1008583,https://www.musinsa.com/app/goods/1008583,퍼스텝(PERSTEP) [패키지] 피그먼트 크루 반팔 8종 패키지 MSST4258,https://image.msscdn.net/images/goods_img/2019...,/opt/ml/fashion-image-search/storage/images/To...
...,...,...,...,...,...
5035,997738,https://www.musinsa.com/app/goods/997738,트릴리온(TRILLION) 컬러 그라데이션 오버핏 반팔티셔츠 BLACK,https://image.msscdn.net/images/goods_img/2019...,/opt/ml/fashion-image-search/storage/images/To...
5036,997781,https://www.musinsa.com/app/goods/997781,트릴리온(TRILLION) 컬러 그라데이션 오버핏 반팔티셔츠 WHITE,https://image.msscdn.net/images/goods_img/2019...,/opt/ml/fashion-image-search/storage/images/To...
5037,997783,https://www.musinsa.com/app/goods/997783,트릴리온(TRILLION) 컬러 그라데이션 오버핏 반팔티셔츠 BLUE,https://image.msscdn.net/images/goods_img/2019...,/opt/ml/fashion-image-search/storage/images/To...
5038,999577,https://www.musinsa.com/app/goods/999577,휠라(FILA) 헤리티지 세리프 로고 루즈핏 반팔티 (FE2RSE5101XOWH),https://image.msscdn.net/images/goods_img/2019...,/opt/ml/fashion-image-search/storage/images/To...


In [113]:
# window 경로 -> mac 경로 변환 함수
def W2M(path: str):
    NEW_PATH = "/opt/ml/fashion-image-search/storage/images/Top" # 저장소
    path = path.split("/")[-1] #xxx.png
    path = os.path.join(NEW_PATH, path)
    return path

# apply 적용해서 수정
def f(df: pd.DataFrame):
    # ./fashion-image-search/back/infra/metadata/save5000/img/Top/252020.png
    df['path'] = df['path'].apply(W2M)
    return df

meta_data = f(meta_data)

meta_data.to_csv("/opt/ml/fashion-image-search/storage/csv/Top.csv", index=False)

#### DataFrame에서 특정 속성에서 특정 값들을 갖는 행을 삭제

In [4]:
df = pd.DataFrame({'key': [100, 200, 300, 400, 500], 'value': [1, 2, 3, 4, 5]})
df

,key,value
0,100,1
1,200,2
2,300,3
3,400,4
4,500,5


In [8]:
%%time
keys_to_delete = [100, 200, 300, 400, 500]
df = df[~df['key'].isin(keys_to_delete)]
df['key'].to_list()

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 1.06 ms


[]

#### make pickle

In [48]:
import pickle
import os

def MakePickle(path: str):
    SAVE_DIR = "./fashion-image-search/back/infra/metadata/pickle"
    
    if not os.path.exists(SAVE_DIR):
        os.makedirs(SAVE_DIR)
    
    # embedding = fclip.encode_images([path], batch_size=1).flatten() # (1, 512) -> flatten -> (512,)
    # embedding /= np.linalg.norm(embedding) # 정규화
    
    key = path.split("\\")[-1][:-4]
    
    pickle_path = os.path.join(SAVE_DIR,f"{key}.pkl")

    # with open(os.path.join(SAVE_DIR,f"{key}.pkl"), "wb") as f:
    #     pickle.dump(embedding, f)
    
    # pickle_path = os.path.join(SAVE_DIR,f"{key}.pkl")
                          
    return pickle_path


meta_data['embedding_path'] = meta_data['path'].apply(MakePickle)

In [ ]:
def MakePickle(self, df: list) -> list[str]:
    SAVE_DIR = "/opt/ml/fashion-image-search/storage/images"
    
    if not os.path.exists(SAVE_DIR):
        os.makedirs(SAVE_DIR)
        
    paths = df['image_path'].tolist()
    keys = df['key'].tolist()
    
    embedding = self.flip.encode_images([paths], batch_size=1).flatten() # (1, 512) -> flatten -> (512,)
    embedding /= np.linalg.norm(embedding) # 정규화
    
    pickles = []

    for key, output_emb in zip(keys,embedding):
        with open(os.path.join(SAVE_DIR,f"{key}.pkl"), "wb") as f:
            pickles.append(os.path.join(SAVE_DIR,f"{key}.pkl"))
            pickle.dump(output_emb, f)
                        
    return pickles

In [ ]:
meta_data

In [19]:
def makeJsonList(csv):
    json_list = []
    for i in tqdm(range(len(meta_data['key']))):
        json_template = { "primary_key": list(csv['key'])[i],
                          "name": list(csv['name'])[i],
                          "link" : {
                                'musinsa_link': list(csv['link'])[i],
                                'image_link': list(csv['image_link'])[i]
                          },
                          "info": {
                                'type': 'Top',
                                'color': 'black'
                          },
                          "path": {
                                 'imaga_path': list(csv['path'])[i],
                                 'pickle_path': list(csv['pickle_path'])[i]
                          }
                          }
        json_list.append(json_template)
        
    return json_list

In [4]:
df = meta_data
jsonList = []
for _, row in df.iterrows():
    json_template = {
        **row,
        "category": {
            "type": "Top"
        }
    }
    jsonList.append(json_template)

In [6]:
from pprint import pprint
pprint(jsonList[:3])

[{'category': {'type': 'Top'},
  'image_link': 'https://image.msscdn.net/images/goods_img/20190401/1000696/1000696_10_500.gif',
  'key': 1000696,
  'link': 'https://www.musinsa.com/app/goods/1000696',
  'name': '어커버(ACOVER) [세트상품]포인트 오버핏 롱티셔츠',
  'path': '/opt/ml/fashion-image-search/storage/images/Top/1000696.png'},
 {'category': {'type': 'Top'},
  'image_link': 'https://image.msscdn.net/images/goods_img/20190402/1001714/1001714_1_500.jpg',
  'key': 1001714,
  'link': 'https://www.musinsa.com/app/goods/1001714',
  'name': '엠엠엘지(MMLG) [Mmlg] MMLG HF-T (BLACK)',
  'path': '/opt/ml/fashion-image-search/storage/images/Top/1001714.png'},
 {'category': {'type': 'Top'},
  'image_link': 'https://image.msscdn.net/images/goods_img/20190408/1008039/1008039_16871513087312_500.jpg',
  'key': 1008039,
  'link': 'https://www.musinsa.com/app/goods/1008039',
  'name': '아노블리어(ANOBLIR) 루즈핏 오픈카라 텐셀 린넨 셔츠 [오트밀]',
  'path': '/opt/ml/fashion-image-search/storage/images/Top/1008039.png'}]
